In [220]:
import pandas

df_train = pandas.read_csv('./train.csv', quotechar="\'")
df_test = pandas.read_csv('./test.csv', quotechar="\'")

In [221]:
import re
import string
import demoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer

demoji.download_codes()
def clean_tweets(data):
    new_data = data.copy()

    username_hash = r'[#@]\w+'
    punctuation = '[%s]+' % re.escape(string.punctuation)
    special_char = r'[^0-9a-zA-Z\s]+'
    number = r'[0-9]+'
    space = r'\s{2,}'
    space_begin_end = r'^\s+|\s+$'
    url = r'(https?|www):\/{1,}\w+\W+\w+\/{1,}\w+'
    char_ref = r'&\w+;'

    for i in range(len(new_data)):
        new_data[i] = re.sub(char_ref, ' ', new_data[i])
        new_data[i] = re.sub(username_hash, '', new_data[i])
        new_data[i] = re.sub(url, '', new_data[i])
        new_data[i] = re.sub(punctuation, '', new_data[i])
        new_data[i] = re.sub(number, '', new_data[i])
        new_data[i] = re.sub(space_begin_end, '', new_data[i])
        new_data[i] = re.sub(space, '', new_data[i])
        new_data[i] = demoji.replace(new_data[i], '')
        new_data[i] = re.sub(special_char, '', new_data[i])

    return new_data

def case_fold(data):
    new_data = data.copy()
    return list(map(lambda s: s.lower(), new_data))

def tokenize(data):
    new_data = data.copy()
    return list(map(lambda s: s.split(' '), new_data))

def stem(data):
    new_data = data.copy()
    stemmer = StemmerFactory().create_stemmer()

    return list(map(lambda s: stemmer.stem(s), new_data))

... OK (Got response in 0.47 seconds)
Writing emoji data to /Users/bangkodir/.demoji/codes.json ...
... OK


In [222]:
tweet_train = stem(case_fold(clean_tweets(df_train['tweet'])))
tweet_test = stem(case_fold(clean_tweets(df_test['tweet'])))

In [223]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(use_idf=True)
tweet_train_vect = tfidf_vect.fit_transform(tweet_train)
tweet_test_vect = tfidf_vect.transform(tweet_test)

In [224]:
from sklearn.svm import SVC

svm_c = SVC(kernel='linear')
svm_c.fit(tweet_train_vect, df_train['category'])

pred = svm_c.predict(tweet_test_vect)

pos = list(filter(lambda s: s == 'positif', pred))
neg = list(filter(lambda s: s == 'negatif', pred))

total = len(df_test['tweet'])
pos_percent = (len(pos) / total) * 100
neg_percent = (len(neg) / total) * 100

print(pos_percent)
print(neg_percent)

25.865447419986936
74.13455258001306


In [225]:
# TODO : test our model here